In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import RobustScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

data = pd.read_csv('creditcard.csv')

dataset = data.to_numpy().astype(np.float32)

scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(25.0, 75.0), unit_variance=False)
dataset[:, [0, 29]] = scaler.fit_transform(dataset[:, [0, 29]])

X = dataset[:, :-1]
y = dataset[:, -1]

# Split the data into 70% training and 30% testing

In [2]:
data_train, data_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the Autoencoder model

In [3]:
# -----------------------------
# 3. Define the Autoencoder Model
# -----------------------------
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        # Encoder: 31 -> 23 -> 19 -> 17 -> 8 with dropout after first two hidden layers
        self.encoder = nn.Sequential(
            nn.Linear(30, 23),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(23, 19),
            nn.Tanh(),
            nn.Dropout(0.2),
            nn.Linear(19, 17),
            nn.Tanh(),
            nn.Linear(17, 8)
        )
        # Decoder: 8 -> 17 -> 19 -> 23 -> 31 with dropout after first two layers
        self.decoder = nn.Sequential(
            nn.Linear(8, 17),
            nn.Tanh(),
            nn.Dropout(0.2),
            nn.Linear(17, 19),
            nn.Tanh(),
            nn.Dropout(0.1),
            nn.Linear(19, 23),
            nn.Tanh(),
            nn.Linear(23, 30)
        )

    def forward(self, x):
        latent = self.encoder(x)
        reconstructed = self.decoder(latent)
        return reconstructed

In [4]:
# Get fraud and non-fraud data using y_train
fraud_indices = np.where(y_train == 1)[0]
non_fraud_indices = np.where(y_train == 0)[0]

fraud_data = data_train[fraud_indices]
non_fraud_data = data_train[non_fraud_indices]

# Calculate number of fraud samples for later use
n = len(fraud_data)
print(f"Number of fraud samples in training data: {n}")

# Select an equal number of non-fraud samples
non_fraud_data_limited = non_fraud_data[:n]

# Combine the selected data
X_balanced = np.vstack((non_fraud_data_limited, fraud_data))
y_balanced = np.hstack((np.zeros(n), np.ones(n)))  # Create labels: 0 for non-fraud, 1 for fraud

# Shuffle the data
shuffle_idx = np.random.permutation(len(X_balanced))
X_balanced_shuffled = X_balanced[shuffle_idx]
y_balanced_shuffled = y_balanced[shuffle_idx]

# Split into training and test sets for SVM
data_train_SVM, data_test_SVM, labels_train_SVM, labels_test_SVM = train_test_split(X_balanced_shuffled, y_balanced_shuffled, test_size=0.2, random_state=42)

# Train SVM model for validation in the oversampling process
model = SVC(kernel='linear', C=1.0)
model.fit(data_train_SVM, labels_train_SVM)

# Get predictions and accuracy
labels_pred_test = model.predict(data_test_SVM)
labels_pred_train = model.predict(data_train_SVM)
accuracy_train = np.mean(labels_pred_train == labels_train_SVM)
accuracy_test = np.mean(labels_pred_test == labels_test_SVM)

print(f"Train Accuracy: {accuracy_train:.4f}")
print(f"Test Accuracy: {accuracy_test:.4f}")

Number of fraud samples in training data: 356
Train Accuracy: 0.9490
Test Accuracy: 0.9021


In [5]:
def get_fraud_indices(labels):
    """Return indices for which label == 1."""
    return np.where(labels == 1)[0]

def get_legit_indices(labels):
    """Return indices for which label == 0."""
    return np.where(labels == 0)[0]

# Example: placeholders for data_train, y_train
# data_train = np.random.randn(100, 30)
# y_train    = np.random.randint(0, 2, size=100)

# Make your copies to oversample
data_train_oversampled = data_train.copy()
labels_train_oversampled = y_train.copy()

# Load your pre-trained autoencoder
autoencoderModel = Autoencoder()
autoencoderModel.load_state_dict(
    torch.load("model/autoencoder/autoencoder30.pth", map_location=torch.device('cpu'))
)
autoencoderModel.eval()

alpha = 0.9

while len(get_fraud_indices(labels_train_oversampled)) < len(get_legit_indices(labels_train_oversampled)):
    fraud_indices = get_fraud_indices(labels_train_oversampled)

    # Check if we have fraud data to work with
    if len(fraud_indices) == 0:
        print("No fraud data found. Cannot proceed with oversampling.")
        break

    # 1) Gather all fraud samples (shape: (num_fraud, 30))
    fraud_only_X = data_train_oversampled[fraud_indices, :]
    fraud_data_tensor = torch.tensor(fraud_only_X, dtype=torch.float32)

    # 2) Encode to latent space
    latent_representations = autoencoderModel.encoder(fraud_data_tensor)
    n = len(latent_representations)
    if n == 0:
        print("Error: no latent representations. Check input data.")
        break

    # 3) Interpolate in latent space + add noise
    noisy_vectors = []
    for _ in range(300):
        i = np.random.randint(0, n)
        j = np.random.randint(0, n)

        z_i = latent_representations[i]
        z_j = latent_representations[j]

        lambda_ = np.random.rand()
        z_ij = lambda_ * z_i + (1 - lambda_) * z_j

        noise = torch.normal(mean=0.0, std=alpha, size=z_ij.shape)
        z_syn = z_ij + noise  # shape (8,)

        noisy_vectors.append(z_syn.detach().cpu().numpy())

    noisy_vectors = np.array(noisy_vectors)  # (300, 8)

    # 4) Decode them to original 30-dim space
    X_synthetic = autoencoderModel.decoder(
        torch.tensor(noisy_vectors, dtype=torch.float32)
    ).detach().cpu().numpy()  # shape (300, 30)

    # 5) Classify them with your SVM model
    #    "model" is your trained SVM that does model.predict(...)
    y_pred = model.predict(X_synthetic)  # shape (300,)

    # 6) Keep only predicted fraud
    synthetic_fraud_mask = (y_pred == 1)
    X_synthetic_fraud = X_synthetic[synthetic_fraud_mask]
    y_synthetic_fraud = y_pred[synthetic_fraud_mask]

    # 7) Append these new fraud samples to oversampled data
    data_train_oversampled = np.vstack((data_train_oversampled, X_synthetic_fraud))
    labels_train_oversampled = np.hstack((labels_train_oversampled, y_synthetic_fraud))

    n_fraud = len(get_fraud_indices(labels_train_oversampled))
    n_legit = len(get_legit_indices(labels_train_oversampled))
    print(f"New synthetic data generated: {len(X_synthetic_fraud)}")
    print(f"Number of fraud in the balanced dataset: {n_fraud} / {n_legit}")

print("Final shape of data:", data_train_oversampled.shape)
print("Final # Fraud:", len(get_fraud_indices(labels_train_oversampled)))
print("Final # Legit:", len(get_legit_indices(labels_train_oversampled)))


New synthetic data generated: 299
Number of fraud in the balanced dataset: 655 / 199008
New synthetic data generated: 299
Number of fraud in the balanced dataset: 954 / 199008
New synthetic data generated: 300
Number of fraud in the balanced dataset: 1254 / 199008
New synthetic data generated: 300
Number of fraud in the balanced dataset: 1554 / 199008
New synthetic data generated: 298
Number of fraud in the balanced dataset: 1852 / 199008
New synthetic data generated: 298
Number of fraud in the balanced dataset: 2150 / 199008
New synthetic data generated: 299
Number of fraud in the balanced dataset: 2449 / 199008
New synthetic data generated: 298
Number of fraud in the balanced dataset: 2747 / 199008
New synthetic data generated: 299
Number of fraud in the balanced dataset: 3046 / 199008
New synthetic data generated: 300
Number of fraud in the balanced dataset: 3346 / 199008
New synthetic data generated: 300
Number of fraud in the balanced dataset: 3646 / 199008
New synthetic data gene

# Train the ALSTM model

In [42]:
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attention_weights = nn.Linear(hidden_size, 1, bias=False)
        
    def forward(self, lstm_output):
        # Check if we need to add a sequence dimension
        if len(lstm_output.shape) == 2:
            # Add a sequence dimension of length 1
            lstm_output = lstm_output.unsqueeze(1)
            
        # Now lstm_output should be (batch_size, seq_len, hidden_size)
        attention_scores = self.attention_weights(lstm_output)  # (batch_size, seq_len, 1)
        attention_scores = attention_scores.squeeze(-1)  # (batch_size, seq_len)
        
        # Apply softmax over sequence dimension
        attention_weights = F.softmax(attention_scores, dim=1)  # (batch_size, seq_len)
        
        # Apply attention weights
        context = torch.sum(lstm_output * attention_weights.unsqueeze(-1), dim=1)  # (batch_size, hidden_size)
        
        return context

class ALSTM(nn.Module):
    def __init__(self, input_size=30, hidden_size=10):
        super().__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=1,
            batch_first=True
        )

        self.attention = Attention(hidden_size)
        
        self.fc = nn.Linear(hidden_size, 1)
        
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        lstm_out, (h, c) = self.lstm(x)
        
        # apply attention: 
        # (be sure your actual attention method expects lstm_out,h,c in that order)
        context = self.attention(lstm_out)  # shape: (batch_size, hidden_size)
        
        # pass through linear layer
        logits = self.fc(context)  # shape: (batch_size, 1)
        
        # apply sigmoid
        out = self.sigmoid(logits) # shape: (batch_size, 1)
        
        return out

In [40]:
def train_model(model, X, y, num_epochs=100, batch_size=32, lr=0.001):
    # define loss function and optimizer
    criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
    optimizer = optim.Adam(model.parameters(), lr=lr)  # Adam optimizer

    # Convert numpy arrays to PyTorch tensors
    X_tensor = torch.FloatTensor(X)
    y_tensor = torch.FloatTensor(y).reshape(-1, 1)  # Ensure proper shape

    if X_tensor.dim() == 2:
        X_tensor = X_tensor.unsqueeze(1)

    dataset = TensorDataset(X_tensor, y_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # train the model
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0.0

        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()
            
            # Forward pass
            outputs = model(batch_X)
            
            # Compute loss - both should be batch_size x 1
            loss = criterion(outputs, batch_y)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        # Print progress
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(dataloader):.4f}')
    
    return model

# Gradient Boosting Classifier

**Input:** Training set $T = \{(x_i, y_i)\}_{i=1}^m$, ALSTM as base learner, number of estimators $(n\_estimators)$

Initialize $h^0(x)$ with a constant: $h^0(x) = \arg \min \sum_{i=1}^n L(y_i, 0)$

**for** $t = 1, \ldots, n\_estimators$ **do**

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Compute residuals: $r_i^t = -\frac{\partial L}{\partial h^{t-1}(x_i)}$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Train $ALSTM_t(x)$ using the dataset $(x_i, r_i^t)$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Calculate step length: $\alpha^t = \arg \min_\alpha \sum_{i=1}^n L(y_i, h^{t-1}(x_i) + \alpha h^t(x_i))$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Update the model: $h(x) = h(x) + \alpha^t h_t(x)$

**end for**

**Output:** Final ensemble model $h(x)$

In [ ]:
def GB_classifier(X, y, n_estimators=10):
    estimators = []

    loss_fn = lambda y, yhat: 1/2 * np.sum((y - yhat)**2)
    
    hx = np.argmin(loss_fn(y_i, 0) for y_i in y)

    for t in range(n_estimators):
        # compute the residual
        residuals = y - hx

        # train model using the dataset (x_i, residuals_i)
        model_t = ALSTM(input_size=30, hidden_size=10)

        train_model(model_t, X, residuals, num_epochs=2, batch_size=32)

        # predict the residuals
        model_t.eval()
        with torch.no_grad():
            # Convert input data to PyTorch tensor
            X_tensor = torch.FloatTensor(X)
            # Get model predictions
            predictions = model_t(X_tensor)
            # Convert predictions to numpy array
            h_t = predictions.numpy().flatten()
        
        # compute step length
        alpha_t = np.argmin(loss_fn(y, hx + alpha * h_t))
        
        # update the predictions
        hx += alpha_t * h_t

        estimators.append((model_t, alpha_t))

    return estimators

In [ ]:
estimators = GB_classifier(data_train_oversampled, labels_train_oversampled, n_estimators=6) 

Epoch [1/2], Loss: 0.0175
Epoch [2/2], Loss: 0.0018
Epoch [1/2], Loss: 0.0168
Epoch [2/2], Loss: 0.0019
Epoch [1/2], Loss: 0.0191
Epoch [2/2], Loss: 0.0018
Epoch [1/2], Loss: 0.0179
Epoch [2/2], Loss: 0.0019
Epoch [1/2], Loss: 0.0189
Epoch [2/2], Loss: 0.0018
Epoch [1/2], Loss: 0.0168
Epoch [2/2], Loss: 0.0019


NameError: name 'ensemble' is not defined

In [44]:
base_value = 0.5  # or store it somewhere else

hx = np.full(X.shape[0], base_value, dtype=float)
for (m, alpha) in estimators:
    hx += alpha * predict_model(m, X)

# threshold at 0.5 for classification
preds = (hx > 0.5).astype(int)
accuracy = np.mean(preds == y_data)
print("Final training accuracy:", accuracy)

NameError: name 'predict_model' is not defined